# sPyTial for Z3

Traditional Z3 work is purely textual - you write constraints, get satisfiable/unsatisfiable results, and examine models through print statements. sPyTial transforms this by making the structure of your constraints, models, and solving process spatially visible.


In [1]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from z3 import *
from spytial import diagram, orientation, group, atomColor, attribute




Let's start with a basic problem and compare traditional vs sPyTial approaches.

In [2]:
# Traditional Z3 approach
x, y = Ints('x y')
solver = Solver()
solver.add(x + y == 10, x > 0, y > 0, x < 8)

print(f"Constraints: {solver.assertions()}")
result = solver.check()
print(f"Result: {result}")

if result == sat:
    model = solver.model()
    print(f"Model: {model}")
    print(f"x = {model[x]}, y = {model[y]}")
else:
    print("No solution found")
    

Constraints: [x + y == 10, x > 0, y > 0, x < 8]
Result: sat
Model: [y = 3, x = 7]
x = 7, y = 3


In [3]:
# sPyTial approach - visualize the model structure.
## THis is actually far less


if result == sat:
    model = solver.model()
    print("Z3 Model Structure:")

    # This is really not a great visualization of the model
    diagram(model, method="inline")


Z3 Model Structure:


## Domain Specific Customization

Since `sPyTial` doesn't know *how* to relationalize a `ModelObj`, we register a relationalizer that does so.


### Shortcomings & trade-offs of this relationalizer.

While relationalizers can be non-trivial, they really need only be implemented once by a domain expert to enable all subsequent spytial diagrams.


*This relationalizer (though large) does a reasonable job for demonstration purposes, but one could imagine an even more sophisticated one for Z3 models*.






- No totalization by default
We only emit explicit function entries and array stores (plus default). If a function’s domain is finite (uninterpreted sorts with universes) and you want a total graph, you’d need to cross-product universes and call m.eval(f(*args), model_completion=True). That’s omitted here to avoid blow-ups; add it behind a “safe size” guard if you need it.

- Datatype selector edges are minimal.
Z3’s Python API around datatypes/accessors can be version-fragile. The code leaves selector emission commented—easy to enable, but you should test against your Z3 version.

- Arrays with symbolic defaults
We detect a single constant default via is_const_array/is_k. If the default is itself complex or depends on variables (rare in models), we just attach it as a label/value atom; we don’t expand lambdas/as-array metaviews further.

- Overloaded names
Decl atoms are qualified by arity (Decl::<name>/<arity>). If you have true overloads with same name/arity but different sorts, collide-proofing would require adding sort signatures (e.g., f/2[Int×Int→Int]).

- Identity and sharing
We memo by (value.hash(), sort.name()) (fallback (str(value), sort.name())). This keeps identical AstRefs unified. If your model reuses equal-but-distinct values that print identically but should be different nodes, you’ll need a different identity policy.

- Large models / performance
We avoid enumerating anything implicit. If you enable totalization or deep datatype traversal, you must set domain caps (e.g., “max 200 tuples/relation”) to keep the graph usable.

- Strings/seqs escaping
We only newline-escape labels. If your renderer needs stricter escaping (quotes, RTL, etc.), extend _escape_label.



In [7]:
from typing import Any, List, Tuple, Optional, Dict
import z3
from spytial import RelationalizerBase, Atom, Relation, relationalizer

@relationalizer(priority=100)
class Z3ModelRelationalizer(RelationalizerBase):
    """
    Z3 ModelRef → (Atoms, Relations), strictly avoiding Python truthiness on Z3 ASTs
    and propagating types from context (expected_sort thunks).
    """

    def can_handle(self, obj: Any) -> bool:
        return (z3 is not None) and isinstance(obj, z3.z3.ModelRef)

    def relationalize(self, obj: Any, walker_func=None) -> Tuple[List[Atom], List[Relation]]:
        assert self.can_handle(obj), "Z3ModelRelationalizer can only handle z3.ModelRef"
        self._reset_state()

        m: z3.ModelRef = obj

        # 1) Pre-seed finite universes for uninterpreted sorts (if provided)
        self._materialize_universes(m)

        # 2) Decl atoms for all decls; Const atoms + defines edge for arity-0
        for decl in m.decls():
            decl_atom = self._make_decl_atom(decl)
            if decl.arity() == 0:
                c_id = self._const_atom_id(decl.name())
                self._add_atom(c_id, "Const", decl.name())
                self._add_edge("defines", c_id, decl_atom.id)

        # 3) Emit constants and function entries/defaults
        for decl in m.decls():
            if decl.arity() == 0:
                self._emit_constant(m, decl)
            else:
                self._emit_function(m, decl)

        atoms_list = list(self._atoms_by_id.values())
        rels_list = self._relations[:]
        # Optional safety: ensure nothing symbolic leaks out
        self._assert_pure_python(atoms_list, rels_list)
        return atoms_list, rels_list

    def _reset_state(self):
        self._atoms_by_id: Dict[str, Atom] = {}
        self._val_to_atom: Dict[Any, str] = {}
        self._relations: List[Relation] = []

    def _add_atom(self, id_: str, type_: str, label: str) -> Atom:
        if id_ in self._atoms_by_id:
            return self._atoms_by_id[id_]
        a = Atom(id=id_, type=type_, label=label)
        self._atoms_by_id[id_] = a
        return a

    def _add_edge(self, name: str, src: str, dst: str):
        # FIX: Use correct Relation constructor format
        self._relations.append(Relation(name=name, atoms=[src, dst]))

    # Sort helpers (no truthiness on Z3 ASTs)
    @staticmethod
    def _sort_name(s: "z3.SortRef") -> str: 
        return s.name()
    
    @staticmethod
    def _is_uninterp_sort(s: "z3.SortRef") -> bool: 
        return s.kind() == z3.Z3_UNINTERPRETED_SORT
    
    @staticmethod
    def _is_array_sort(s: "z3.SortRef") -> bool: 
        return s.kind() == z3.Z3_ARRAY_SORT
    
    @staticmethod
    def _is_int_sort(s: "z3.SortRef") -> bool: 
        return s.kind() == z3.Z3_INT_SORT
    
    @staticmethod
    def _is_real_sort(s: "z3.SortRef") -> bool: 
        return s.kind() == z3.Z3_REAL_SORT
    
    @staticmethod
    def _is_bv_sort(s: "z3.SortRef") -> bool: 
        return s.kind() == z3.Z3_BV_SORT
    
    @staticmethod
    def _is_bool_sort(s: "z3.SortRef") -> bool: 
        return s.kind() == z3.Z3_BOOL_SORT
    
    @staticmethod
    def _is_string_sort(s: "z3.SortRef") -> bool: 
        return s.name() == "String"
    
    @staticmethod
    def _is_seq_sort(s: "z3.SortRef") -> bool: 
        return s.kind() == z3.Z3_SEQ_SORT

    def _collect_candidate_sorts(self, m: "z3.ModelRef") -> List["z3.SortRef"]:
        acc = set()
        for d in m.decls():
            acc.add(d.range())
            for i in range(d.arity()):
                acc.add(d.domain(i))
        return list(acc)

    def _materialize_universes(self, m: "z3.ModelRef"):
        sorts = self._collect_candidate_sorts(m)
        for s in sorts:
            if not self._is_uninterp_sort(s):
                continue
            try:
                univ = m.get_universe(s)
            except Exception:
                univ = None
            if (univ is None) or (len(univ) == 0):
                continue
            for elem in univ:
                _ = self._value_atom(elem, expected_sort=s)

    def _decl_atom_id(self, decl: "z3.FuncDeclRef") -> str:
        return f"Decl::{decl.name()}/{decl.arity()}"

    def _const_atom_id(self, name: str) -> str:
        return f"Const::{name}"

    def _make_decl_atom(self, decl: "z3.FuncDeclRef") -> Atom:
        dom = [decl.domain(i).name() for i in range(decl.arity())]
        rng = decl.range().name()
        label = f"{decl.name()}({', '.join(dom)}) → {rng}"
        return self._add_atom(self._decl_atom_id(decl), "Decl", label)

    def _emit_constant(self, m: "z3.ModelRef", decl: "z3.FuncDeclRef"):
        c_id = self._const_atom_id(decl.name())
        expected = decl.range()

        v = None
        try:
            v = m[decl]
        except Exception:
            try:
                v = m.eval(decl())
            except Exception:
                v = None

        if v is None:
            return

        val_id = self._value_atom(v, expected_sort=expected)
        self._add_edge("value_of", c_id, val_id)

        srt = v.sort()
        if self._is_array_sort(srt):
            self._emit_array_store_graph(base_name=decl.name(), arr=v, parent_const_id=c_id)

    def _emit_function(self, m: "z3.ModelRef", decl: "z3.FuncDeclRef"):
        d_id = self._decl_atom_id(decl)
        interp = m[decl]

        if isinstance(interp, z3.FuncInterpRef):
            n = interp.num_entries()
            for idx in range(n):
                entry = interp.entry(idx)
                app_id = f"App::{decl.name()}/{idx}"
                self._add_atom(app_id, "App", f"{decl.name()}@{idx}")
                self._add_edge("of", app_id, d_id)

                # Args typed by domain sorts
                arg_count = entry.num_args()
                for ai in range(arg_count):
                    arg_v = entry.arg(ai)
                    arg_sort = decl.domain(ai)
                    arg_id = self._value_atom(arg_v, expected_sort=arg_sort)
                    self._add_edge(f"arg_{ai}", app_id, arg_id)

                # Result typed by range sort
                res_id = self._value_atom(entry.value(), expected_sort=decl.range())
                self._add_edge("result", app_id, res_id)

            # Default (else) value, if present
            else_v = None
            try:
                else_v = interp.else_value()
            except Exception:
                else_v = None
            if else_v is not None:
                else_id = self._value_atom(else_v, expected_sort=decl.range())
                self._add_edge("default", d_id, else_id)

    def _emit_array_store_graph(self, base_name: str, arr: "z3.AstRef", parent_const_id: Optional[str]):
        entries: List[Tuple[z3.AstRef, z3.AstRef]] = []
        t = arr
        default_val = None

        # Walk nested Stores
        while hasattr(z3, "is_store") and z3.is_store(t):
            base = t.arg(0)
            key = t.arg(1)
            val = t.arg(2)
            entries.append((key, val))
            t = base

        # Detect ConstArray default
        is_const_default = False
        if hasattr(z3, "is_const_array") and z3.is_const_array(t):
            is_const_default = True
        elif hasattr(z3, "is_k") and z3.is_k(t):
            is_const_default = True
        if is_const_default:
            default_val = t.arg(1)

        arr_sort = arr.sort()
        key_sort = arr_sort.domain()
        val_sort = arr_sort.range()

        # Emit stores in chronological order
        for i, (k, v) in enumerate(reversed(entries)):
            app_id = f"App::{base_name}_map/{i}"
            self._add_atom(app_id, "App", f"{base_name}_map@{i}")
            if parent_const_id is not None:
                self._add_edge("of", app_id, parent_const_id)
            key_id = self._value_atom(k, expected_sort=key_sort)
            val_id = self._value_atom(v, expected_sort=val_sort)
            self._add_edge("key", app_id, key_id)
            self._add_edge("val", app_id, val_id)

        # Default (if any)
        if (default_val is not None) and (parent_const_id is not None):
            def_id = self._value_atom(default_val, expected_sort=val_sort)
            self._add_edge(f"{base_name}_default", parent_const_id, def_id)

    def _value_atom(self, v: "z3.AstRef", expected_sort: Optional["z3.SortRef"] = None) -> str:
        key = self._value_key(v)
        if key in self._val_to_atom:
            return self._val_to_atom[key]

        actual = v.sort()
        # CRITICAL FIX: Use explicit None check instead of 'or' with Z3 objects
        srt = expected_sort if expected_sort is not None else actual

        # Bool
        if self._is_bool_sort(srt):
            if z3.is_true(v):
                aid, label = "Bool::true", "true"
            elif z3.is_false(v):
                aid, label = "Bool::false", "false"
            else:
                label = str(v)
                aid = f"Bool::{label}"
            self._add_atom(aid, "Bool", label)

        # Int
        elif self._is_int_sort(srt):
            sval = self._to_int_str(v)
            aid = f"Int::{sval}"
            self._add_atom(aid, "Int", sval)

        # Real
        elif self._is_real_sort(srt):
            sval = self._to_real_str(v)
            aid = f"Real::{sval}"
            self._add_atom(aid, "Real", sval)

        # Bit-vectors
        elif self._is_bv_sort(srt):
            width = srt.size()
            sval = self._to_bv_uint_str(v, width)
            aid = f"BV[{width}]::{sval}"
            self._add_atom(aid, f"BV[{width}]", sval)

        # Strings / Seqs
        elif self._is_string_sort(srt):
            sval = self._escape_label(v.as_string() if hasattr(v, "as_string") else str(v))
            aid = f"String::{sval}"
            self._add_atom(aid, "String", sval)
        elif self._is_seq_sort(srt):
            sval = self._escape_label(str(v))
            aid = f"Seq::{sval}"
            self._add_atom(aid, "Seq", sval)

        # Uninterpreted elements
        elif self._is_uninterp_sort(srt):
            label = str(v)
            aid = f"U::{self._sort_name(srt)}::{label}"
            self._add_atom(aid, self._sort_name(srt), label)

        # Arrays
        elif self._is_array_sort(srt):
            label = f"Array[{srt.domain().name()}→{srt.range().name()}]"
            aid = f"Array::{self._stable_hash(label, v)}"
            self._add_atom(aid, "Array", label)

        # Datatypes / tuples
        elif srt.kind() == z3.Z3_DATATYPE_SORT:
            ctor_name = v.decl().name() if z3.is_app(v) else self._sort_name(srt)
            label = ctor_name
            aid = f"DT::{self._sort_name(srt)}::{self._stable_hash(label, v)}"
            self._add_atom(aid, self._sort_name(srt), label)

        # Fallback
        else:
            label = self._escape_label(str(v))
            aid = f"{self._sort_name(srt)}::{self._stable_hash(label, v)}"
            self._add_atom(aid, self._sort_name(srt), label)

        self._val_to_atom[key] = aid
        return aid

    def _assert_pure_python(self, atoms: List[Atom], rels: List[Relation]):
        """Ensure no Z3 objects leak across the boundary."""
        if z3 is None:
            return
        def _is_z3(x: Any) -> bool:
            return isinstance(x, (z3.AstRef, z3.FuncDeclRef, z3.SortRef, z3.ModelRef))
        for a in atoms:
            if _is_z3(a.id) or _is_z3(a.type) or _is_z3(a.label):
                raise TypeError(f"Leaked Z3 object in Atom: {a}")
        for r in rels:
            if _is_z3(r.name):
                raise TypeError(f"Leaked Z3 object in Relation: {r}")
            for atom_id in r.atoms:
                if _is_z3(atom_id):
                    raise TypeError(f"Leaked Z3 object in Relation atoms: {r}")

    def _value_key(self, v: "z3.AstRef") -> Any:
        try:
            return (v.hash(), v.sort().name())
        except Exception:
            return (str(v), v.sort().name())

    @staticmethod
    def _to_int_str(v: "z3.AstRef") -> str:
        if hasattr(z3, "is_int_value") and z3.is_int_value(v):
            return str(v.as_long())
        if hasattr(z3, "is_rational_value") and z3.is_rational_value(v):
            try:
                num = v.numerator_as_long()
                den = v.denominator_as_long()
                if den == 1:
                    return str(num)
            except Exception:
                pass
        s = str(v).strip()
        if s.startswith("-"):
            s2 = s[1:]
            if s2.isdigit():
                return s
        elif s.isdigit():
            return s
        try:
            sv = z3.simplify(v)
            if hasattr(z3, "is_int_value") and z3.is_int_value(sv):
                return str(sv.as_long())
        except Exception:
            pass
        return s

    @staticmethod
    def _to_real_str(v: "z3.AstRef") -> str:
        if hasattr(z3, "is_rational_value") and z3.is_rational_value(v):
            num = v.numerator_as_long()
            den = v.denominator_as_long()
            return f"{num}/{den}" if den != 1 else f"{num}"
        try:
            return v.as_decimal(20).rstrip("?")
        except Exception:
            return str(v)

    @staticmethod
    def _to_bv_uint_str(v: "z3.AstRef", width: int) -> str:
        try:
            return str(z3.BV2Int(v, is_signed=False).as_long())
        except Exception:
            try:
                sv = z3.simplify(v)
                return str(z3.BV2Int(sv, is_signed=False).as_long())
            except Exception:
                return str(v)

    @staticmethod
    def _escape_label(s: str) -> str:
        return s.replace("\n", "\\n")

    @staticmethod
    def _stable_hash(head: str, v: Any) -> str:
        base = f"{head}|{str(v)}"
        h = 0
        for ch in base:
            h = (h * 131 + ord(ch)) & 0xFFFFFFFF
        return f"{h:08x}"

In [5]:
mr = Z3ModelRelationalizer().relationalize(model, None)
print(mr)

([Atom(id='Decl::y/0', type='Decl', label='y() → Int'), Atom(id='Const::y', type='Const', label='y'), Atom(id='Decl::x/0', type='Decl', label='x() → Int'), Atom(id='Const::x', type='Const', label='x'), Atom(id='Int::3', type='Int', label='3'), Atom(id='Int::7', type='Int', label='7')], [Relation(name='defines', atoms=['Const::y', 'Decl::y/0']), Relation(name='defines', atoms=['Const::x', 'Decl::x/0']), Relation(name='value_of', atoms=['Const::y', 'Int::3']), Relation(name='value_of', atoms=['Const::x', 'Int::7'])])


In [6]:
diagram(model)